In [ ]:
---
title: "04장 범주형 데이터 다루기"
date: 2022/11/10
date-modified: last-modified
---

범주형 데이터는 순서가 있는 것과 없는 것을 구분해야 한다.

영화 평점처럼 순서가 있는 레이블로 예측하는 것은 사이킷런에서 지원하지 않는다. 

[Mord 파이썬 패키지](https://github.com/fabianp/mord){target=_blank} 참고

In [13]:
import pandas as pd
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


## 순서 있는 특성 매핑


In [2]:
size_mapping = {'XL':3, 'L':2, 'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


### 인코딩한 특성을 원래의 범주형으로 되돌리기

인코딩을 위해 만든 딕셔너리의 키, 밸류를 거꾸로 한 딕셔너리 만듦

In [3]:
inv_size_maping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_maping)

0     M
1     L
2    XL
Name: size, dtype: object

## 클래스 레이블 인코딩

### 직접 정수로 바꾸기

1. np.unique()를 이용하여 레이블의 고유값 반환
1. enumerate 인덱스와 값을 거꾸로 하여 값을 키로, 인덱스를 값으로 하는 딕셔너리 만들기

In [5]:
import numpy as np
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
print('레이블 인코딩', class_mapping)
df['classlabel'] = df['classlabel'].map(class_mapping)
df

레이블 인코딩 {'class1': 0, 'class2': 1}


,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


### LabelEncoder 사용하기

In [14]:
from sklearn.preprocessing import LabelEncoder

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

### 정수 레이블을 원본으로 되돌리기

In [15]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

### 여러 열을 한 번에 인코딩

- LabelEncoder는 입력 데이터로 1차원 배열을 기대한다.
- 여러 열을 한 번에 정수로 변환하려면 범주형 데이터를 정수로 인코딩하는 OrdinalEncoder와 데이터프레임의 열마다 변환을 도와주는 ColumnTrnsformer를 이용한다.

In [17]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

ord_enc = OrdinalEncoder(dtype=int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])

X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

## 순서 없는 특성에 원-핫 인코딩 적용

### sklearn OneHotEncoder 사용

In [23]:
# 하나의 특성만 인코딩
from sklearn.preprocessing import OneHotEncoder

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']

X = df[['color','size','price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [24]:
# 여러 개의 특성 인코딩

from sklearn.compose import ColumnTransformer
X = df[['color','size','price']].values
c_trans = ColumnTransformer([('onehot', OneHotEncoder(), [0, 1]),
                             ('nothing', 'passthrough', [2])])
c_trans.fit_transform(X)

array([[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 10.1],
       [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 13.5],
       [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.3]], dtype=object)

### pd.get_dummies

- 원핫인코딩된 데이터를 사용할 때는 다중 공선성 주의!!! 
- 이를 테면 역행렬 계산이 어려움.
- 변수 간 상관관계를 줄이려면 특성 열 하나 삭제 -> drop_first=True

In [29]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,color_blue,color_green,color_red,size_L,size_M,size_XL
0,10.1,0,1,0,0,1,0
1,13.5,0,0,1,1,0,0
2,15.3,1,0,0,0,0,1


OneHotEncoder에서 중복 열 삭제 -> drop='first', categories='auto'

In [31]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
            ('onehot', color_ohe, [0, 1]),
            ('nothing', 'passthrough', [2])])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1.0, 0.0, 10.1],
       [0.0, 1.0, 0.0, 0.0, 13.5],
       [0.0, 0.0, 0.0, 1.0, 15.3]], dtype=object)

- [머신 러닝의 모델 평가와 모델 선택, 알고리즘 선택 – 1장. 기초](https://tensorflow.blog/%eb%a8%b8%ec%8b%a0-%eb%9f%ac%eb%8b%9d%ec%9d%98-%eb%aa%a8%eb%8d%b8-%ed%8f%89%ea%b0%80%ec%99%80-%eb%aa%a8%eb%8d%b8-%ec%84%a0%ed%83%9d-%ec%95%8c%ea%b3%a0%eb%a6%ac%ec%a6%98-%ec%84%a0%ed%83%9d-1/){target=_blank}
- [머신 러닝의 모델 평가와 모델 선택, 알고리즘 선택 – 2장. 부트스트래핑과 불확실성](https://tensorflow.blog/%eb%a8%b8%ec%8b%a0-%eb%9f%ac%eb%8b%9d%ec%9d%98-%eb%aa%a8%eb%8d%b8-%ed%8f%89%ea%b0%80%ec%99%80-%eb%aa%a8%eb%8d%b8-%ec%84%a0%ed%83%9d-%ec%95%8c%ea%b3%a0%eb%a6%ac%ec%a6%98-%ec%84%a0%ed%83%9d-2/){target=_blank}
- [머신 러닝의 모델 평가와 모델 선택, 알고리즘 선택 – 3장. 크로스밸리데이션과 하이퍼파라미터 튜닝](https://tensorflow.blog/%eb%a8%b8%ec%8b%a0-%eb%9f%ac%eb%8b%9d%ec%9d%98-%eb%aa%a8%eb%8d%b8-%ed%8f%89%ea%b0%80%ec%99%80-%eb%aa%a8%eb%8d%b8-%ec%84%a0%ed%83%9d-%ec%95%8c%ea%b3%a0%eb%a6%ac%ec%a6%98-%ec%84%a0%ed%83%9d-3/){target=_blank}